# Granularity
Standardise the data such that they have the same granularity.  

 * Weekly from Week #17 2012
 * Locale : Singapore

Import modules:-

In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re
from datetime import date

### Set directory paths:-

In [2]:
raw_path = "../../../data/1_raw/"
out_path = "../../../data/2_interim/"
clean_path = "../../../data/3_clean/"

if not os.path.exists(out_path):
    os.makedirs(out_path)
if not os.path.exists(clean_path):
    os.makedirs(clean_path)

### Compile weather data into Weekly, Singapore average format

In [3]:
weather_files = glob.glob(raw_path+'weather/*.csv')
len(weather_files)

1231

#### Create DataFrame from file:

In [4]:
dfWeather = pd.read_csv(weather_files.pop(), encoding='latin_1')
for f in weather_files:
    df = pd.read_csv(f, encoding='latin_1')
    dfWeather = dfWeather.append(df)

Use DataFrame describe and head to inspect data:

In [5]:
dfWeather.describe()

,Year,Month,Day
count,37456.000000,37456.000000,37456.000000
mean,2014.599584,6.310231,15.725117
std,1.807743,3.482746,8.798235
min,2012.000000,1.000000,1.000000
25%,2013.000000,3.000000,8.000000
50%,2015.000000,6.000000,16.000000
75%,2016.000000,9.000000,23.000000
max,2018.000000,12.000000,31.000000


In [6]:
dfWeather.head()

,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 Min Rainfall (mm),Highest 60 Min Rainfall (mm),Highest 120 Min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
0,Marina Barrage,2013,5,1,0,,,,29.1,32.3,27.7,3.2,32
1,Marina Barrage,2013,5,2,0,,,,29.5,33.3,27.1,4.2,20.9
2,Marina Barrage,2013,5,3,0.4,,,,29,31.5,27.5,3.6,20.9
3,Marina Barrage,2013,5,4,0,,,,29.6,32.7,27,5.1,28.1
4,Marina Barrage,2013,5,5,0.4,,,,29.1,33.2,27.3,4.3,21.2


#### Remove unecessary columns:

In [7]:
dfWeather.columns

Index(['Station', 'Year', 'Month', 'Day', 'Daily Rainfall Total (mm)',
       'Highest 30 Min Rainfall (mm)', 'Highest 60 Min Rainfall (mm)',
       'Highest 120 Min Rainfall (mm)', 'Mean Temperature (°C)',
       'Maximum Temperature (°C)', 'Minimum Temperature (°C)',
       'Mean Wind Speed (km/h)', 'Max Wind Speed (km/h)'],
      dtype='object')

Use drop() to remove four columns

In [8]:
dfWeather = dfWeather.drop(['Station', 'Highest 30 Min Rainfall (mm)', 
                            'Highest 60 Min Rainfall (mm)',
                           'Highest 120 Min Rainfall (mm)'], axis=1)

#### Rename column headers into usable labels, i.e. standard ASCII UTF-8 characters.

In [9]:
dfWeather.columns

Index(['Year', 'Month', 'Day', 'Daily Rainfall Total (mm)',
       'Mean Temperature (°C)', 'Maximum Temperature (°C)',
       'Minimum Temperature (°C)', 'Mean Wind Speed (km/h)',
       'Max Wind Speed (km/h)'],
      dtype='object')

In [10]:
dfWeather.columns = ['Year', 'Month', 'Day', 'Daily Rainfall Total',
       'Mean Temperature', 'Maximum Temperature',
       'Minimum Temperature', 'Mean Wind Speed',
       'Max Wind Speed']

In [11]:
dfWeather.columns

Index(['Year', 'Month', 'Day', 'Daily Rainfall Total', 'Mean Temperature',
       'Maximum Temperature', 'Minimum Temperature', 'Mean Wind Speed',
       'Max Wind Speed'],
      dtype='object')

#### Deal with missing / invalid data

In [12]:
dfWeather = dfWeather.fillna(np.nan)

In [13]:
for col in dfWeather:
    dfWeather[col] = pd.to_numeric(dfWeather[col], errors='coerce')

In [14]:
dfWeather.describe()

,Year,Month,Day,Daily Rainfall Total,Mean Temperature,Maximum Temperature,Minimum Temperature,Mean Wind Speed,Max Wind Speed
count,37456.000000,37456.000000,37456.000000,36659.000000,34910.000000,35819.000000,35805.000000,36474.000000,36040.000000
mean,2014.599584,6.310231,15.725117,6.220677,27.894343,31.599056,25.227248,8.213256,32.679584
std,1.807743,3.482746,8.798235,13.206875,1.175741,1.674631,1.504022,3.831799,9.631011
min,2012.000000,1.000000,1.000000,0.000000,22.200000,22.800000,0.000000,1.800000,0.000000
25%,2013.000000,3.000000,8.000000,0.000000,27.100000,30.500000,24.100000,5.700000,26.300000
50%,2015.000000,6.000000,16.000000,0.200000,27.900000,31.800000,25.100000,7.200000,31.300000
75%,2016.000000,9.000000,23.000000,5.800000,28.800000,32.800000,26.300000,9.700000,37.100000
max,2018.000000,12.000000,31.000000,208.200000,31.700000,36.800000,29.800000,45.000000,138.600000


#### Deal with improbable data:

In [15]:
dfWeather['Minimum Temperature'] = dfWeather['Minimum Temperature'].apply(lambda x: np.nan if x==0 else x)

In [16]:
dfWeather.describe()

,Year,Month,Day,Daily Rainfall Total,Mean Temperature,Maximum Temperature,Minimum Temperature,Mean Wind Speed,Max Wind Speed
count,37456.000000,37456.000000,37456.000000,36659.000000,34910.000000,35819.000000,35802.000000,36474.000000,36040.000000
mean,2014.599584,6.310231,15.725117,6.220677,27.894343,31.599056,25.229361,8.213256,32.679584
std,1.807743,3.482746,8.798235,13.206875,1.175741,1.674631,1.486250,3.831799,9.631011
min,2012.000000,1.000000,1.000000,0.000000,22.200000,22.800000,20.000000,1.800000,0.000000
25%,2013.000000,3.000000,8.000000,0.000000,27.100000,30.500000,24.100000,5.700000,26.300000
50%,2015.000000,6.000000,16.000000,0.200000,27.900000,31.800000,25.100000,7.200000,31.300000
75%,2016.000000,9.000000,23.000000,5.800000,28.800000,32.800000,26.300000,9.700000,37.100000
max,2018.000000,12.000000,31.000000,208.200000,31.700000,36.800000,29.800000,45.000000,138.600000


#### Get week number:

In [17]:
def get_weeknum(row):
    isocal = date(int(row['Year']), int(row['Month']), int(row['Day'])).isocalendar()
    weeknum = str(isocal[0]) + '-W' + "%02d"%isocal[1] 
    return weeknum

In [18]:
dfWeather['Y-Week'] = dfWeather.apply(lambda row: get_weeknum(row), axis=1)

In [19]:
dfWeather.head()

,Year,Month,Day,Daily Rainfall Total,Mean Temperature,Maximum Temperature,Minimum Temperature,Mean Wind Speed,Max Wind Speed,Y-Week
0,2013,5,1,0.0,29.1,32.3,27.7,3.2,32.0,2013-W18
1,2013,5,2,0.0,29.5,33.3,27.1,4.2,20.9,2013-W18
2,2013,5,3,0.4,29.0,31.5,27.5,3.6,20.9,2013-W18
3,2013,5,4,0.0,29.6,32.7,27.0,5.1,28.1,2013-W18
4,2013,5,5,0.4,29.1,33.2,27.3,4.3,21.2,2013-W18


#### Recalculate data by week number:

In [20]:
group_dict = {'Minimum Temperature':['min'], 
              'Daily Rainfall Total':['mean'], 
              'Mean Temperature':['mean'], 
              'Mean Wind Speed':['mean'], 
              'Maximum Temperature':['max'], 
              'Max Wind Speed':['max']}

In [21]:
groupedWeather = dfWeather.groupby(['Y-Week']).agg(group_dict)


In [22]:
groupedWeather.head()

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed
,min,mean,mean,mean,max,max
Y-Week,,,,,,
2011-W52,24.5,0.529412,27.468750,7.705882,33.4,35.3
2012-W01,23.0,1.860504,27.002632,11.973950,32.9,114.1
2012-W02,21.8,3.473109,26.391453,8.773109,33.7,64.8
2012-W03,21.4,12.357983,27.158772,6.971429,34.6,60.8
2012-W04,22.0,1.522689,26.740000,9.651261,33.2,51.8


In [23]:
groupedWeather.columns

MultiIndex(levels=[['Minimum Temperature', 'Daily Rainfall Total', 'Mean Temperature', 'Mean Wind Speed', 'Maximum Temperature', 'Max Wind Speed'], ['max', 'mean', 'min']],
           labels=[[0, 1, 2, 3, 4, 5], [2, 1, 1, 1, 0, 0]])

#### Rename columns:

In [24]:
groupedWeather.columns = [col[0] for col in groupedWeather.columns]

In [25]:
groupedWeather.columns

Index(['Minimum Temperature', 'Daily Rainfall Total', 'Mean Temperature',
       'Mean Wind Speed', 'Maximum Temperature', 'Max Wind Speed'],
      dtype='object')

In [26]:
groupedWeather.head()

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed
Y-Week,,,,,,
2011-W52,24.5,0.529412,27.468750,7.705882,33.4,35.3
2012-W01,23.0,1.860504,27.002632,11.973950,32.9,114.1
2012-W02,21.8,3.473109,26.391453,8.773109,33.7,64.8
2012-W03,21.4,12.357983,27.158772,6.971429,34.6,60.8
2012-W04,22.0,1.522689,26.740000,9.651261,33.2,51.8


In [27]:
groupedWeather.rename(index=str, columns={'Daily Rainfall Total':'Mean Daily Rainfall Total'})

,Minimum Temperature,Mean Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed
Y-Week,,,,,,
2011-W52,24.5,0.529412,27.468750,7.705882,33.4,35.3
2012-W01,23.0,1.860504,27.002632,11.973950,32.9,114.1
2012-W02,21.8,3.473109,26.391453,8.773109,33.7,64.8
2012-W03,21.4,12.357983,27.158772,6.971429,34.6,60.8
2012-W04,22.0,1.522689,26.740000,9.651261,33.2,51.8
2012-W05,21.3,7.144538,26.499160,9.527731,32.8,65.5
2012-W06,21.6,1.165546,27.468103,10.646218,34.4,48.2
2012-W07,22.1,5.373109,26.780508,7.770588,33.8,59.4
2012-W08,22.5,4.483193,27.568644,8.307563,34.2,55.1


#### Save to file:
Save file to interim folder

In [28]:
groupedWeather.to_csv(out_path+'weather.csv')

# Dengue and Malaria data
Read the infectious disease data:

In [29]:
dfDisease = pd.read_csv(raw_path+"weekly-infectious-disease-bulletin-cases.csv", index_col=0)

Keep only the Malaria, Dengue Fever and Dengue Haemorrhagic Fever data

In [30]:
dfDisease = dfDisease[(dfDisease['disease']=='Malaria') |
                      (dfDisease['disease']=='Dengue Fever') | 
                      (dfDisease['disease']=='Dengue Haemorrhagic Fever')]

In [31]:
dfDisease.head()

,disease,no._of_cases
epi_week,,
2012-W01,Dengue Fever,74
2012-W01,Dengue Haemorrhagic Fever,0
2012-W01,Malaria,1
2012-W02,Dengue Fever,64
2012-W02,Dengue Haemorrhagic Fever,2


### Pivoting
Use pivot() to get data by week:

In [32]:
dfDisease = dfDisease.pivot(columns = 'disease', values = 'no._of_cases')

Combine Dengue Fever and Dengue Haemorrhagic Fever into one column

In [33]:
dfDisease['Dengue'] = dfDisease['Dengue Fever']+dfDisease['Dengue Haemorrhagic Fever']

And remove unnecessary columns

In [34]:
dfDisease = dfDisease.drop(['Dengue Fever', 'Dengue Haemorrhagic Fever'], axis=1)

#### Save to file

In [35]:
dfDisease.to_csv(out_path+'disease.csv')

# Combine Data
Combine the two data sets into one DataFrame

In [36]:
dfData = pd.concat([groupedWeather, dfDisease], axis=1)

In [37]:
dfData.head()

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed,Malaria,Dengue
2011-W52,24.5,0.529412,27.468750,7.705882,33.4,35.3,NaN,NaN
2012-W01,23.0,1.860504,27.002632,11.973950,32.9,114.1,1.0,74.0
2012-W02,21.8,3.473109,26.391453,8.773109,33.7,64.8,2.0,66.0
2012-W03,21.4,12.357983,27.158772,6.971429,34.6,60.8,2.0,61.0
2012-W04,22.0,1.522689,26.740000,9.651261,33.2,51.8,3.0,52.0


In [38]:
dfData[-5:]

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed,Malaria,Dengue
2018-W09,23.1,5.868750,27.675000,9.875455,35.7,46.4,NaN,NaN
2018-W10,23.1,4.117857,27.622619,10.873636,35.6,70.2,NaN,NaN
2018-W11,22.8,1.581250,28.268293,9.970000,35.5,66.6,NaN,NaN
2018-W12,22.0,3.099107,28.148193,9.647273,35.2,53.6,NaN,NaN
2018-W13,20.5,5.661702,26.869697,7.927778,33.8,133.2,NaN,NaN


#### Remove invalid rows with no data:

In [39]:
dfData = dfData.dropna(subset = ['Malaria','Dengue'], how='all')

In [40]:
dfData.describe()

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed,Malaria,Dengue
count,312.000000,312.000000,312.000000,312.000000,312.000000,312.000000,313.000000,313.000000
mean,22.438141,6.254222,27.915425,8.177514,34.477564,70.079167,1.386581,231.341853
std,0.770885,4.420455,0.844697,1.653113,0.904122,16.445666,1.556854,179.699575
min,20.000000,0.000000,25.701695,5.339496,31.000000,41.400000,0.000000,24.000000
25%,22.000000,3.018082,27.301500,6.990965,33.900000,57.500000,0.000000,81.000000
50%,22.400000,5.390739,27.929241,7.848179,34.500000,68.600000,1.000000,205.000000
75%,22.900000,8.672663,28.471849,9.019923,35.000000,79.300000,2.000000,316.000000
max,24.500000,22.285714,30.032203,14.556190,36.800000,138.600000,9.000000,891.000000


In [41]:
dfData.head()

,Minimum Temperature,Daily Rainfall Total,Mean Temperature,Mean Wind Speed,Maximum Temperature,Max Wind Speed,Malaria,Dengue
2012-W01,23.0,1.860504,27.002632,11.973950,32.9,114.1,1.0,74.0
2012-W02,21.8,3.473109,26.391453,8.773109,33.7,64.8,2.0,66.0
2012-W03,21.4,12.357983,27.158772,6.971429,34.6,60.8,2.0,61.0
2012-W04,22.0,1.522689,26.740000,9.651261,33.2,51.8,3.0,52.0
2012-W05,21.3,7.144538,26.499160,9.527731,32.8,65.5,1.0,85.0


### Reformat indexing

In [42]:
YearWeek = [[int(x) for x in item.split('-W')] for item in dfData.index]


In [43]:
dfYearWeek = pd.DataFrame.from_records(YearWeek, index = dfData.index, columns = ['Year', 'Week'])

In [44]:
dfDataYW = pd.concat([dfData,dfYearWeek], axis=1)

In [45]:
dfDataYW = dfDataYW.set_index(['Year','Week'])

In [46]:
dfDataYW.head()

Minimum Temperature  Daily Rainfall Total  Mean Temperature  \
Year Week                                                                
2012 1                    23.0              1.860504         27.002632   
     2                    21.8              3.473109         26.391453   
     3                    21.4             12.357983         27.158772   
     4                    22.0              1.522689         26.740000   
     5                    21.3              7.144538         26.499160   

           Mean Wind Speed  Maximum Temperature  Max Wind Speed  Malaria  \
Year Week                                                                  
2012 1           11.973950                 32.9           114.1      1.0   
     2            8.773109                 33.7            64.8      2.0   
     3            6.971429                 34.6            60.8      2.0   
     4            9.651261                 33.2            51.8      3.0   
     5            9.527731                 32.8            65.5      1.0   

           Dengue  
Year Week          
2012 1       74.0  
     2       66.0  
     3       61.0  
     4       52.0  
     5       85.0

#### Save to file
Save combined data frame into clean folder

In [47]:
dfDataYW.to_csv(clean_path+'data.csv')